In [6]:
using DataFrames
using Distributions
using Gadfly
using StochMCMC
Gadfly.push_theme(:dark)

srand(123);

# Define data parameters
w0 = -.3; w1 = -.5; stdev = 5.;
a = 1 / stdev;

# Generate Hypothetical Data
n = 200;
x = rand(Uniform(-1, 1), n);
A = [ones(length(x)) x];
B = [w0; w1];
f = A * B;
y = f + rand(Normal(0, alpha), n);

my_df = DataFrame(Independent = round(x, 4), Dependent = round(y, 4));

In [9]:
function logprior(theta::Array{Float64}; mu::Array{Float64} = zero_vec, s::Array{Float64} = s)
  w0_prior = log(pdf(Normal(mu[1, 1], s[1, 1]), theta[1]))
  w1_prior = log(pdf(Normal(mu[2, 1], s[2, 2]), theta[2]))
   w_prior = [w0_prior w1_prior]

  return w_prior |> sum
end

"""
The log likelihood function is given by the following codes:
"""
function loglike(theta::Array{Float64}; alpha::Float64 = a, x::Array{Float64} = x, y::Array{Float64} = y)
  yhat = theta[1] + theta[2] * x

  likhood = Float64[]
  for i in 1:length(yhat)
    push!(likhood, log(pdf(Normal(yhat[i], alpha), y[i])))
  end

  return likhood |> sum
end

"""
The log posterior function is given by the following codes:
"""
function logpost(theta::Array{Float64})
  loglike(theta, alpha = a, x = x, y = y) + logprior(theta, mu = zero_vec, s = eye_mat)
end

logpost

In [10]:
# Hyperparameters
zero_vec = zeros(2)
eye_mat = eye(2)

2×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0

In [11]:
mh_object = MH(logpost; init_est = [.1; .1]);
chain1 = mcmc(mh_object, r = 10000);

In [12]:
burn_in = 100
thinning = 10

# Expetation of the Posterior
est1 = mapslices(mean, chain1[(burn_in + 1):thinning:end, :], [1]);
est1

1×2 Array{Float64,2}:
 -0.30074  -0.503438